In [ ]:
import sys
# package_path = '../input/pytorch-image-models/pytorch-image-models-master' #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
# sys.path.append(package_path)

# import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

# sys.path.append('../input/pytorchimagemodels')

# efficientnet_path='../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'

# sys.path.append(efficientnet_path)

In [ ]:
efficientnet_path='../input/efficientnet-pytorch10/EfficientNet-PyTorch-1.0'
# ../input/efficientnet-pytorch10/EfficientNet-PyTorch-1.0
sys.path.append(efficientnet_path)

In [ ]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler
from functools import partial

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss

In [ ]:
# tf ns effnet and ViT
CFG = {
    'fold_num': 5,
    'n_classes': 5,
    'seed': 719,
    'merged_data':True,
    'evaluate': True,
    'submit': True,
    'multi_model': True,
    'model_arch': 'tf_efficientnet_b3_ns', # 'seresnext101_32x4d', #'tf_efficientnet_b3_ns', #'efficientnet-b3', #'tf_efficientnet_b3_ns',
    'model_arch2': 'vit_base_patch16_384', # vit_base_patch16_384 deit_base_patch16_384 'seresnext101_32x4d', #'tf_efficientnet_b3_ns', #'efficientnet-b3', #'tf_efficientnet_b3_ns',
    'model_arch3': 'deit_base_patch16_384',
    'model_arch4': 'resnext50_32x4d', 
    'model_arch5': 'seresnext101_32x4d', 
    'img_size': 512, #512, #256,
    'img_size2': 384,
    'img_size3': 384,
    'img_size4': 512,
    'epochs': 50,
    'train_bs': 8, #64,
    'valid_bs': 32,
    'T_0': 10,
    'lr': 1e-5,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'weight_dirs': ['../input/cassava-efficientnet/best_score_effnet_b3_ls03_v2', 
                    '../input/cassava-efficientnet/best_score_resnext50_data_v2', 
                    '../input/cassava-efficientnet/best_score_vit_p16_ls03_data_v2',
                    ], #'../input/cassava-efficientnet/best_score_vit_p16_ls03','../input/cassava-efficientnet/best_score_tf_ns_ls_5folds'# best_score_seresnext101_data_v2 best_score_resnext50_data_v2'' '../input/cassava-efficientnet/best_score_vit_p16_ls03'], # best_score_tf_ns_ls_5folds'], #best_score_seresnext101_ls03_agb4', ../input/cassava-efficientnet/best_score_tf_ns_ls_5folds best_score_vit_p32_ls03
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 7, # best 6
    'weights': [1,1,1,1,1,1,1,1],
    'fold_weights': [1,1,1,1,1] #[1,1,1,1]
}

In [ ]:
# # deit
# CFG = {
#     'fold_num': 5,
#     'n_classes': 5,
#     'seed': 719,
#     'merged_data':True,
#     'evaluate': True,
#     'multi_model': False,
#     'model_arch': 'deit_base_patch16_384', # 'seresnext101_32x4d', #'tf_efficientnet_b3_ns', #'efficientnet-b3', #'tf_efficientnet_b3_ns',
#     'model_arch2': 'deit_base_patch16_384', # vit_base_patch16_384 'seresnext101_32x4d', #'tf_efficientnet_b3_ns', #'efficientnet-b3', #'tf_efficientnet_b3_ns',
#     'img_size': 384, #512, #256,
#     'img_size2': 384,
#     'epochs': 50,
#     'train_bs': 8, #64,
#     'valid_bs': 32,
#     'T_0': 10,
#     'lr': 1e-5,
#     'min_lr': 1e-6,
#     'weight_decay':1e-6,
#     'num_workers': 4,
#     'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
#     'weight_dirs': ['../input/cassava-efficientnet/best_score_deit_p16_ls03_data_v2_ep10'], #best_score_deit_p16_ls03'], best_score_deit_p16_ls03 best_score_tf_ns_ls_5folds, '../input/cassava-efficientnet/best_score_vit_p16_ls03'], #['../input/cassava-efficientnet/best_score_tf_ns_bitempered02','../input/cassava-efficientnet/best_score_tf_ns_ls_5folds'], #best_score_seresnext101_ls03_agb4', ../input/cassava-efficientnet/best_score_tf_ns_ls_5folds best_score_vit_p32_ls03
#     'verbose_step': 1,
#     'device': 'cuda:0',
#     'tta': 4,
#     'weights': [1,1,1,1,1],
#     'fold_weights': [1,1,1,1,1] #[1,1,1,1]
# }

In [ ]:
# # seresnext101_32x4d
# CFG = {
#     'fold_num': 5,
#     'n_classes': 5,
#     'seed': 719,
#     'merged_data':True,
#     'evaluate': True,
#     'multi_model': False,
#     'model_arch': 'resnext50_32x4d', # 'seresnext101_32x4d', #'tf_efficientnet_b3_ns', #'efficientnet-b3', #'tf_efficientnet_b3_ns',
#     'model_arch2': 'vit_base_patch16_384', # vit_base_patch16_384 deit_base_patch16_384 'seresnext101_32x4d', #'tf_efficientnet_b3_ns', #'efficientnet-b3', #'tf_efficientnet_b3_ns',
#     'model_arch3': 'deit_base_patch16_384',
#     'model_arch4': 'resnext50_32x4d', 
#     'img_size': 512, #512, #256,
#     'img_size2': 384,
#     'img_size3': 384,
#     'img_size4': 512,
#     'vit_img_size': 384,
#     'epochs': 50,
#     'train_bs': 8, #64,
#     'valid_bs': 32,
#     'T_0': 10,
#     'lr': 1e-5,
#     'min_lr': 1e-6,
#     'weight_decay':1e-6,
#     'num_workers': 4,
#     'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
#     'weight_dirs': ['../input/cassava-efficientnet/best_score_resnext50_data_v2'], #best_score_seresnext101_ls03_agb4',
#     'verbose_step': 1,
#     'device': 'cuda:0',
#     'tta': 4,
#     'weights': [1,1,1,1,1],
#     'fold_weights': [1,1,1,1,1] #[1,1,1,1]
# }

In [ ]:
# # vit
# CFG = {
#     'fold_num': 5,
#     'n_classes': 5,
#     'seed': 719,
#     'merged_data':True,
#     'evaluate': True,
#     'multi_model': False,
#     'model_arch': 'vit_base_patch16_384', # 'seresnext101_32x4d', #'tf_efficientnet_b3_ns', #'efficientnet-b3', #'tf_efficientnet_b3_ns',
#     'model_arch2': None, # 'seresnext101_32x4d', #'tf_efficientnet_b3_ns', #'efficientnet-b3', #'tf_efficientnet_b3_ns',
#     'img_size': 384, #512, #256,
#     'img_size2': 384,
#     'epochs': 50,
#     'train_bs': 32, #64,
#     'valid_bs': 32,
#     'T_0': 10,
#     'lr': 1e-5,
#     'min_lr': 1e-6,
#     'weight_decay':1e-6,
#     'num_workers': 4,
#     'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
#     'weight_dirs': ['../input/cassava-efficientnet/best_score_vit_p16_ls03_data_v2_ep10'], # ../input/cassava-efficientnet/best_score_vit_p16_ls03_data_v2'], #best_score_vit_p32_ls03'],
#     'verbose_step': 1,
#     'device': 'cuda:0',
#     'tta': 4,
#     'weights': [1,1,1,1,1],
#     'fold_weights': [1,1,1,1,1] #[1,1,1,1]
# }

> We could do stratified validation split in each fold to make each fold's train and validation set looks like the whole train set in target distributions.

# Helper Functions

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

# img = get_img('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
# plt.imshow(img)
# plt.show()

# Dataset

In [ ]:
class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

# Define Train\Validation Image Augmentations

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2


def get_train_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms(img_size=CFG['img_size']):
    return Compose([
            CenterCrop(img_size, img_size, p=1.),
            Resize(img_size, img_size),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_inference_transforms(img_size=CFG['img_size']):
    return Compose([
            RandomResizedCrop(img_size, img_size),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# def get_inference_transforms(img_size=CFG['img_size']):
#     return Compose([
#             RandomResizedCrop(img_size, img_size),
#             Transpose(p=0.5),
#             HorizontalFlip(p=0.5),
#             VerticalFlip(p=0.5),
#             HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
#             RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
#             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
#             ToTensorV2(p=1.0),
#         ], p=1.)

# Model

In [ ]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
from timm.models.vision_transformer import VisionTransformer, _cfg
from timm.models.registry import register_model

# @register_model
# def deit_base_patch16_224(pretrained=False, **kwargs):
#     model = VisionTransformer(
#         patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
#         norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
#     model.default_cfg = _cfg()
#     if pretrained:
#         checkpoint = torch.hub.load_state_dict_from_url(
#             url="https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth",
#             map_location="cpu", check_hash=True
#         )
#         model.load_state_dict(checkpoint["model"])
#     return model

# self, img_size=224, patch_size=16, in_chans=3, num_classes=1000, embed_dim=768, depth=12,
#                  num_heads=12, mlp_ratio=4., qkv_bias=True, qk_scale=None, representation_size=None,
#                  drop_rate=0., attn_drop_rate=0., drop_path_rate=0., hybrid_backbone=None, norm_layer=None):

@register_model
def deit_base_patch16_384(pretrained=False, **kwargs):
    """ DeiT base model @ 384x384 from paper (https://arxiv.org/abs/2012.12877).
    ImageNet-1k weights from https://github.com/facebookresearch/deit.
    """
#     model_kwargs = dict(patch_size=16, embed_dim=768, depth=12, num_heads=12, **kwargs)
#     model = _create_vision_transformer('vit_deit_base_patch16_384', pretrained=pretrained, **model_kwargs)
#     model = VisionTransformer(patch_size=16, embed_dim=768, depth=12, num_heads=12, **kwargs)
    model = VisionTransformer(
        patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = _cfg()
    return model

In [ ]:
def get_model(model_arch, n_class, pretrained=False):
    if 'tf_' in model_arch and 'ns' in model_arch:
        print('Loading EfficientNet model')
        model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = model.classifier.in_features
        model.classifier = nn.Linear(n_features, n_class)
    elif 'resnext' in model_arch:
        print('Loading ResNext model')
        model = timm.create_model(model_arch, pretrained=pretrained)
#         n_features = model.last_linear.in_features
#         model.last_linear = nn.Linear(n_features, n_class)
        n_features = model.fc.in_features
        model.fc = nn.Linear(n_features, n_class)

    elif 'vit' in model_arch:
        print('Loading ViT model')
        model = timm.create_model(model_arch, pretrained=pretrained)
        model.head = nn.Linear(model.head.in_features, n_class)
    elif 'deit' in model_arch:
        print('Loading DeiT model')
#         model = torch.hub.load('facebookresearch/deit:main', model_arch, pretrained=pretrained)
        model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = model.head.in_features
        model.head = nn.Linear(n_features, n_class)
    else:
        model = EfficientNet.from_name(model_arch)
        if pretrained:
            model.load_state_dict(torch.load(pretrained))
    #         self.efficient_net=EfficientNet.from_pretrained('efficientnet-b3',num_classes=CLASSES)
        in_features=model._fc.in_features
        model._fc=nn.Linear(in_features,n_class)
    return model

# Main Loop

In [ ]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
def predict_valid(train, folds, weight_dirs, class_col_name='label'):
    device = torch.device(CFG['device'])
    val_preds_all = 0
    val_preds_all_2 = 0
    for fold, (trn_idx, val_idx) in enumerate(folds):
#         if fold > 0:
#             break
        print('Validation Inference fold {} started'.format(fold))
   
        val_preds = []
       #TODO: uncomment for loop
        for i, weight_dir in enumerate(weight_dirs): 
            if 'best' in weight_dir or 'basemodel' in weight_dir:
                weight_path = glob(f'{weight_dir}/fold{fold}*')
    #             weight_path = glob(f'{weight_dir}/best_val_score_fold{fold}*')
                weight_path = weight_path[0]
            else:
                weight_path = f'{weight_dir}/best_{mode}_{CFG["model_arch"]}_fold_{fold}.pth'
            
            if multi_model and 'vit' in weight_path:
                img_size = CFG['img_size2']
                model_arch = CFG['model_arch2']
            elif multi_model and 'deit' in weight_path:
                img_size = CFG['img_size3']
                model_arch = CFG['model_arch3']
            elif multi_model and 'seresnext101' in weight_path:
                img_size = CFG['img_size4']
                model_arch = CFG['model_arch5']
            elif multi_model and 'resnext50' in weight_path:
                img_size = CFG['img_size4']
                model_arch = CFG['model_arch4']
            else:
                img_size = CFG['img_size']
                model_arch = CFG['model_arch']
                
            print(f'Loading weight: {weight_path}')

            if 'deit' in weight_path or 'seresnext' in weight_path:
#                 weight_path = '../input/cassava-efficientnet/fold0_epoch0_val_score0.855.pt'
                model = torch.load(weight_path)
                model.cuda()
            else:
                model = get_model(model_arch, CFG['n_classes']).to(device) #CFG['pretrained_path'] n_classes train.label.nunique()
                model.load_state_dict(torch.load(weight_path))
                
            if CFG['merged_data'] and 'v2' in weight_path:
                print('Loading Merged Data.')
                valid_ = train[train['fold'].astype(int)==fold]
                img_path = '../input/cassava-leaf-disease-merged/train/'
            else:
                valid_ = train.loc[val_idx,:].reset_index(drop=True)
                img_path = '../input/cassava-leaf-disease-classification/train_images/'
                
            valid_ds = CassavaDataset(valid_, img_path, 
                                      transforms=get_inference_transforms(img_size), output_label=False)
            val_loader = torch.utils.data.DataLoader(
                    valid_ds, 
                    batch_size=CFG['valid_bs'],
                    num_workers=CFG['num_workers'],
                    shuffle=False,
                    pin_memory=False,
                )

            with torch.no_grad():
                for _ in range(CFG['tta']):
                    val_preds += [CFG['weights'][0]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, val_loader, device)]
        #############################
        val_preds = np.mean(val_preds, axis=0) 
        print('fold {} validation loss = {:.5f}'.format(fold, log_loss(valid_[class_col_name].values, val_preds)))
        print('fold {} validation accuracy = {:.5f}'.format(fold, (valid_[class_col_name].values==np.argmax(val_preds, axis=1)).mean()))
        val_preds_all += (valid_[class_col_name].values==np.argmax(val_preds, axis=1)).mean()
        del model
        torch.cuda.empty_cache()
    fold_score = val_preds_all / CFG['fold_num']
    return print(f'{CFG["fold_num"]} folds mean Accuracy: {fold_score}')


In [ ]:
def predict_test(test, folds, weight_dirs, loaded_state_dict=True):
    device = torch.device(CFG['device'])
    files_path = '../input/cassava-leaf-disease-classification/test_images/'
    test_size = len(os.listdir(files_path))
    test_preds_all = np.zeros((test_size, CFG['n_classes']))
    tst_preds_all = 0
    tst_preds_all_2 = 0
    for fold, (trn_idx, val_idx) in enumerate(folds):
        print('Inference fold {} started'.format(fold))

        tst_preds = []
        for i, weight_dir in enumerate(weight_dirs):

            if 'best' in weight_dir or 'basemodel' in weight_dir:
                weight_path = glob(f'{weight_dir}/fold{fold}*')
                weight_path = weight_path[0]
            else:
                weight_path = f'{weight_dir}/best_{mode}_{CFG["model_arch"]}_fold_{fold}.pth'
                
            if multi_model and 'vit' in weight_path:
                img_size = CFG['img_size2']
                model_arch = CFG['model_arch2']
            elif multi_model and 'deit' in weight_path:
                img_size = CFG['img_size3']
                model_arch = CFG['model_arch3']
            elif multi_model and 'seresnext101' in weight_path:
                img_size = CFG['img_size4']
                model_arch = CFG['model_arch5']
            elif multi_model and 'resnext50' in weight_path:
                img_size = CFG['img_size4']
                model_arch = CFG['model_arch4']
            else:
                img_size = CFG['img_size']
                model_arch = CFG['model_arch']
    
            if 'deit' in weight_path or 'seresnext' in weight_path:
                model = torch.load(weight_path)
                model.cuda()
            else:
                model = get_model(model_arch, CFG['n_classes']).to(device) #CFG['pretrained_path']
                model.load_state_dict(torch.load(weight_path))

            test_ds = CassavaDataset(test, '../input/cassava-leaf-disease-classification/test_images/', 
                                     transforms=get_inference_transforms(img_size), output_label=False)

            tst_loader = torch.utils.data.DataLoader(
                test_ds, 
                batch_size=CFG['valid_bs'],
                num_workers=CFG['num_workers'],
                shuffle=False,
                pin_memory=False,
            )

            with torch.no_grad():
                for _ in range(CFG['tta']):
                    tst_preds += [CFG['weights'][0]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, tst_loader, device)]

        tst_preds = np.mean(tst_preds, axis=0) 
        
        tst_preds_all += tst_preds

        del model
        torch.cuda.empty_cache()

    tst_preds_all /= CFG['fold_num']
    return tst_preds_all

In [ ]:
# if __name__ == '__main__':
seed_everything(CFG['seed'])
 # for training only, need nightly build pytorch
test = pd.DataFrame()
test['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))

if CFG['merged_data']:
    train = pd.read_csv('../input/cassava-2019-2020-dataset/merged_5folds.csv')
    fold_col = 'stratify_group'
    class_col_name = 'class_id'
else:
    train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')  
    fold_col = 'label'
    class_col_name = 'label'
    
folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train.shape[0]), train[fold_col].values)

mode = 'acc'
evaluate = CFG['evaluate']
weight_dirs = CFG['weight_dirs']
multi_model = CFG['multi_model']

if evaluate:
    predict_valid(train, folds, weight_dirs, class_col_name=class_col_name)

In [ ]:
# if __name__ == '__main__':
# seed_everything(CFG['seed'])
 # for training only, need nightly build pytorch
test = pd.DataFrame()
test['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))

if CFG['merged_data']:
    train = pd.read_csv('../input/cassava-2019-2020-dataset/merged_5folds.csv')
    fold_col = 'stratify_group'
    class_col_name = 'class_id'
else:
    train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')  
    fold_col = 'label'
    class_col_name = 'label'

folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train.shape[0]), train[fold_col].values)

mode = 'acc'
weight_dirs = CFG['weight_dirs']
multi_model = CFG['multi_model']

submit = CFG['submit']

if submit:
    tst_preds_all =  predict_test(test, folds, weight_dirs)
    test['label'] = np.argmax(tst_preds_all, axis=1)
    test.to_csv('submission.csv', index=False)